In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # to convert address in latitude and longitude value
# for plotting     
import matplotlib.cm as cm
import matplotlib.colors as colors
# for K-means clustering
from sklearn.cluster import KMeans
# for mapping with folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from sklearn import preprocessing
print('Libraries imported')

In [ ]:

barna_df = df_data_1[df_data_1['City']=='Barcelona']
barna_df = barna_df.reset_index(drop=True)
barna_df.head()

In [ ]:
address    = 'Barcelona, Spain'
geolocator = Nominatim()
location   = geolocator.geocode(address)
latitude   = location.latitude
longitude  = location.longitude

print('The coordinates of Barcelona are {}, {}.'.format(latitude, longitude))

map_barna= folium.Map(location=[latitude,longitude], zoom_start=12)

#adding markers to the Map
for lat, long, label in zip(barna_df['Latitude'], barna_df['Longitude'], barna_df['Place Name']):
    label = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=label,
    colour='blue',
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_barna)

map_barna

In [ ]:

barna_df =barna_df.loc[barna_df['Place Name'].isin(['Barcelona'])]
barna_df =barna_df.reset_index(drop=True)

In [ ]:
barna_df.head()


In [ ]:
barna_df.drop(['State','County','City'],axis=1,inplace=True)
barna_df.shape


In [ ]:
barna_df['Place Name'][0]='El Raval'
barna_df['Place Name'][1]='Gothic Quarter' 
barna_df['Place Name'][2]='Ciutat Vella, La Barcelonata'
barna_df['Place Name'][3]='El Poble Sec' 
barna_df['Place Name'][4]='El Poble Nou'
barna_df['Place Name'][5]='Sant Gervasi, Hospital Plato' 
barna_df['Place Name'][6]='City Center, Passeig de Gracia'
barna_df['Place Name'][7]='Diagonal' 
barna_df['Place Name'][8]='Girona, Jardins de Jaume Perich, Carrer d\'Arago'
barna_df['Place Name'][9]='placa de Tetuan, Urquinaona' 
barna_df['Place Name'][10]='Urgell,Jardin de Cesar Martinell'
barna_df['Place Name'][11]='Vila de Gracia' 
barna_df['Place Name'][12]='Fort Pienc, Sagrada Familia'
barna_df['Place Name'][13]='Sants, Taragona, Placa del Centre' 
barna_df['Place Name'][14]='Sant Antoni, Rocafort'
barna_df['Place Name'][15]='Porta,Virrei Amat, La prosperitat ' 
barna_df['Place Name'][16]='Vallvidrera'
barna_df['Place Name'][17]='Sant Marti, La Verneda, Glories, Arc de Triomf' 
barna_df['Place Name'][18]='El Besos I El Maresme, Paolo Alto Market'
barna_df['Place Name'][19]='La Verneda I La Pau, Sant Marti de Provencals' 
barna_df['Place Name'][20]='Sant Gervasi-Galvany, La Bonanova, Turo Parc'
barna_df['Place Name'][21]='Sant Gervasi-La Bonanova, Av. Tibidabo,  El Putxet' 
barna_df['Place Name'][22]='Vallcarca I Els Penitents, El parc del Turo del Putxet, El Coll'
barna_df['Place Name'][23]='Gracia, Can Baro, Alfons X' 
barna_df['Place Name'][24]='El Baix Guinardo, El Camp D\'En Grassot I Gracia  Nova'
barna_df['Place Name'][25]='El Camp de L\'Arpa del Clot' 
barna_df['Place Name'][26]='La Sagrega'
barna_df['Place Name'][27]='Les Corts, La Maternitat I Sant Ramon, Sants-Montjuic' 
barna_df['Place Name'][28]='La Nova Esquerra de L\'Exiample, Entenca'
barna_df['Place Name'][29]='Sant Adreu, Baro de Viver' 
barna_df['Place Name'][30]='Vilapicina, Can Peguera, El Turo de La Peira'
barna_df['Place Name'][31]='El Carmel, Horta, La Font D\'En Fargues' 
barna_df['Place Name'][32]='La Trinitat Nova, Trinitat Vella, Torre Baro, Vallbona'
barna_df['Place Name'][33]='Saria, Pedralbes, Ciutat Universitaria' 
barna_df['Place Name'][34]='Horta, Sant Genis Dels Agudells, La Teixonera'
barna_df['Place Name'][35]='Hospital Clinic, La Fira' 
barna_df['Place Name'][36]='Verdaguer, Passeig de Sant Joan'
barna_df['Place Name'][37]='Parc de Montjuic, La Marina de Port, La Marina del Prat Vermell' 
barna_df['Place Name'][38]='Port de Barcelona'
barna_df['Place Name'][39]='Zona Franca' 
barna_df['Place Name'][40]='El Guinardo'
barna_df['Place Name'][41]='Les Roquetes'

In [ ]:
barna_df.drop(barna_df.index[42], inplace=True)


In [ ]:
barna_df.drop(barna_df.index[42], inplace=True)


In [ ]:
barna_df.drop(barna_df.index[42], inplace=True)


In [ ]:
barna_df = barna_df.rename(columns={"Place Name": "Neighborhood"})
barna_df.head()

In [ ]:
#creating url and requesiting the API data
neighborhood_lng = barna_df.loc[0, 'Longitude']
neighborhood_lat = barna_df.loc[0, 'Latitude']
radius = 500
Query   = 'coffee'
#create request
url='https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    API_VERSION,
    neighborhood_lat,
    neighborhood_lng,
    radius,
    LIMIT,
    Query)
results = requests.get(url).json()

In [ ]:
def getNearbyvenues( names, latitudes, longitudes, radius = 500 ):
    
    venues_list=[]
    for name, lat, lng, in zip( names, latitudes, longitudes ):
        print(name)
        #create request
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            API_VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        #make get request
        results=requests.get(url).json()["response"]['groups'][0]['items']
      
        #returning only the relevant information
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name'], v['venue']['id'] ) for v in results])
        
        nearby_venues= pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns=['Neighbourhood',
                               'Neighbourhood latitude',
                               'Neighbourhood longitude',
                               'Venue',
                               'Venue Latitude',
                               'Venue Longitude',
                               'Venue category',
                               'Id']
    return(nearby_venues)

In [ ]:
barcelona_venues=getNearbyvenues(names=barna_df['Neighborhood'], latitudes=barna_df['Latitude'],longitudes=barna_df['Longitude'])


In [ ]:
barcelona_venues.head()


In [ ]:
barcelona_venues.groupby('Neighbourhood').count().head() #counting the number of venues in each neighborhood


In [ ]:
# lets get those data of expense and number of ratings
#results = []
#for idt in barcelona_venues['Id']:
#    url='https://api.foursquare.com/v2/venues/'+idt+'?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
#        CLIENT_ID,
#        CLIENT_SECRET,
#        API_VERSION,
#        neighborhood_lat,
#        neighborhood_lng)
#    try:
#        results.append(requests.get(url).json()["response"]['venue']['attributes']['groups'][0]['items'][0]['priceTier'])
#    except:
#        results.append('Na')

In [ ]:
#analysing neighborhoods
barcelona_onehot = pd.get_dummies(barcelona_venues[['Venue category']], prefix = '', prefix_sep='')
barcelona_onehot['Neighbourhood']=barcelona_venues['Neighbourhood']
fixed_columns = [barcelona_onehot.columns[-1]]+list(barcelona_onehot.columns[:-1])
barcelona_onehot= barcelona_onehot[fixed_columns]
barcelona_onehot.shape
#grouping by neighbourhood
barcelona_grouped=barcelona_onehot.groupby('Neighbourhood').mean().reset_index()
barcelona_grouped

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#printing most common venues
num_top_venues=10
for hood in barcelona_grouped['Neighbourhood']:
    print('____'+hood+'____')
    temp=barcelona_grouped[barcelona_grouped['Neighbourhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
#sorting in descending order
num_top_venues=10
indicators=['st','nd','rd']
columns=['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append("{}{} Most Common Venue".format(ind+1,indicators[ind]))
    except:
        columns.append("{}th Most Common Venue".format(ind+1))
#new dataframe
neighbourhoods_venuessorted = pd.DataFrame(columns=columns)
neighbourhoods_venuessorted['Neighbourhood']=barcelona_grouped['Neighbourhood']
for ind in np.arange(barcelona_grouped.shape[0]):
    neighbourhoods_venuessorted.iloc[ind,1:] = return_most_common_venues(barcelona_grouped.iloc[ind, :], num_top_venues)
neighbourhoods_venuessorted

In [ ]:
# Kmeans clustering
Kclusters = 10
barcelona_grouped_clustering= barcelona_grouped.drop('Neighbourhood',1)
#run k means
kmeans=KMeans(n_clusters=Kclusters, random_state=0).fit(barcelona_grouped_clustering)
kmeans.labels_[0:10]

In [ ]:
barcelona_merged=barna_df

barcelona_merged['Cluster Labels']=kmeans.labels_
neighbourhoods_venuessorted.shape
barcelona_merged=barcelona_merged.join(neighbourhoods_venuessorted.set_index('Neighbourhood'), on='Neighborhood')
barcelona_merged

In [ ]:
# visulaize clusters
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=12)
x = np.arange(Kclusters)
ys=[i+x+(i*x)**2 for i in range(Kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#add markers
markers_colors=[]
for lat, lon, poi, cluster in zip(barcelona_merged['Latitude'],barcelona_merged['Longitude'], barcelona_merged['Neighborhood'], barcelona_merged['Cluster Labels']):
    label=folium.Popup(str(poi)+' Cluster '+str(cluster),parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radiesu=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [ ]:
df_data1 = df_data_2[df_data_2.columns[1:3]]
df_data3 = df_data1.rename(columns={'1r Trimestre': 'Price per sq. m'})
df_data3.head()

In [ ]:
df_data3.drop(df_data3.index[0], inplace=True)


In [ ]:
df_data4 = df_data3[df_data3['Price per sq. m'].str.contains("nd") == False] # droppinf rows with NaN or Nd values 
# next step is to select only those rows which partially match the rows of the main barcelona Table
# cleaning the data
df_data4.loc[:,'Barris']= [x.split('.')[1] for x in df_data4['Barris']]

In [ ]:
# adding the datasets manually
df5 = barcelona_merged
df5['Rent per sq m'] = 1
df5['Rent per sq m'][1]  = 19.5
df5['Rent per sq m'][10] = 13.5
df5['Rent per sq m'][13] = 14.1
df5['Rent per sq m'][11] = 13
df5['Rent per sq m'][17] = 13
df5['Rent per sq m'][0]  = 19.8
df5['Rent per sq m'][3]  = 12.7
df5['Rent per sq m'][2]  = 12.3
df5['Rent per sq m'][4]  = 9.5
df5['Rent per sq m'][5]  = 12.9
df5['Rent per sq m'][6]  = 11.2
df5['Rent per sq m'][7]  = 19.5
df5['Rent per sq m'][8]  = 13.4
df5['Rent per sq m'][9]  = 13
df5['Rent per sq m'][11] = 13.5
df5['Rent per sq m'][12] = 12.8
df5['Rent per sq m'][14] = 11.9
df5['Rent per sq m'][15] = 11.2
df5['Rent per sq m'][16] = 13.2
df5['Rent per sq m'][18] = 9.8
df5['Rent per sq m'][19] = 10
df5['Rent per sq m'][20] = 13.2
df5['Rent per sq m'][21] = 13.6
df5['Rent per sq m'][22] = 9.4
df5['Rent per sq m'][23] = 9.9
df5['Rent per sq m'][24] = 11.5
df5['Rent per sq m'][25] = 11.6
df5['Rent per sq m'][26] = 12.4
df5['Rent per sq m'][27] = 12.4
df5['Rent per sq m'][28] = 14.5
df5['Rent per sq m'][29] = 14.6
df5['Rent per sq m'][30] = 15
df5['Rent per sq m'][31] = 13.2
df5['Rent per sq m'][32] = 12.4
df5['Rent per sq m'][33] = 16
df5['Rent per sq m'][34] = 14
df5['Rent per sq m'][35] = 14.1
df5['Rent per sq m'][36] = 10.1
df5['Rent per sq m'][37] = 9.8
df5['Rent per sq m'][38] = 14.9
df5['Rent per sq m'][39] = 11.2
df5['Rent per sq m'][40] = 11.1
df5['Rent per sq m'][41] = 13.7

In [ ]:
df5.head()


In [ ]:
# normalizing the rent column
def standardize( df, label):
    """
    standardizes a series with name ``label'' within the pd.DataFrame
    ``df''.
    """
    df = df.copy(deep=True)
    series = df.loc[:, label]
    avg = series.mean()
    stdv = series.std()
    series_standardized = (series - avg)/ stdv
    return series_standardized
Out = standardize(df5, 'Rent per sq m')
new=[]
for i in df5['Rent per sq m']:
    new.append(Out[i])

In [ ]:
# final result , Hence we can show the similarity or dissimilarity between different neighborhoods alongwith Rent prices, whcih will recommend the person where to get a house
df5['Rent per sq m'] = new

In [ ]:
df5.head()
